In [1]:
!pip install gensim==4.3.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 880.0 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 72.4 MB/s eta 0:00:00
  Created wheel for fst-pso: filename=fst_pso-1.8.1-py3-none-any.whl size=20430 sha256=9ec5f64331f5ddf9a1dcda4b75b24e7b2d809cc09541f25c0c33acc903566816
  Stored in directory: /root/.cache/pip/wheels/69/f5/e5/18ad53fe1ed6b2af9fad05ec052e4acbac8e92441df44bad2e
  Created wheel for miniful: filename=miniful-0.0.6-py3-none-any.whl size=3507 sha256=d96a7af0cdb46df08406e5931d6fd583ce2e7f7bc2c8dd107cf458667cfb3990
  Stored in directory: /root/.ca

In [1]:
import pickle
from gensim.models import Word2Vec
import numpy as np
from google.colab import drive
import os
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/Colab Notebooks/NLP/Task DP-CNN/dp_cnn_tokens.pkl'


# Mở và đọc dữ liệu từ file .pkl
with open(file_path, 'rb') as f:
    data = pickle.load(f)

# Giả sử 'data' chứa các khóa 'train_tokens' và 'test_tokens' như từ điển
train_tokens = data['train_tokens']
test_tokens = data['test_tokens']
val_tokens = data['val_tokens']

Mounted at /content/drive


In [2]:
from gensim.models import Word2Vec

# Tạo danh sách tất cả token từ các file trong tập train
all_sentences = list(train_tokens.values())  # Mỗi file là một danh sách token

# Huấn luyện mô hình Word2Vec
word2vec_model = Word2Vec(sentences=all_sentences, vector_size=30, window=5, min_count=1, workers=4)

# Lấy danh sách token phổ biến nhất trong Word2Vec
top_tokens = list(word2vec_model.wv.index_to_key)[:10]

# Chọn một token bất kỳ để kiểm tra
sample_token = top_tokens[1] if top_tokens else None

if sample_token:
    print(f"Vector của token '{sample_token}':\n{word2vec_model.wv[sample_token]}")

Vector của token '<RETURN>':
[-1.0158436  -0.60524035  1.9724342   0.08814531 -0.3852217  -0.39393735
  1.3229451   0.815394   -1.4410672  -0.39808413  1.660108    0.21830828
  0.4051605  -1.0566111  -0.44229773 -1.4093052   0.42097414  0.67398983
 -2.0631337   1.3903568  -0.9638187   0.10557427 -0.28721297  2.2844765
  1.1519322   1.1712509   0.606368    0.78285456  0.02403784 -2.7176857 ]


In [3]:
import numpy as np

MAX_SEQUENCE_LENGTH = 100  # Số token tối đa mỗi file
EMBEDDING_DIM = 30  # Kích thước vector embedding

def encode_tokens(tokens, max_length=MAX_SEQUENCE_LENGTH):
    """Chuyển token thành vector embedding, padding nếu thiếu"""
    encoded = np.zeros((max_length, EMBEDDING_DIM))  # Ma trận đầu vào
    for i, token in enumerate(tokens[:max_length]):  # Giới hạn max_length
        if token in word2vec_model.wv:
            encoded[i] = word2vec_model.wv[token]
    return encoded

# 🔹 1. Mã hóa dữ liệu train và test
X_train = np.array([encode_tokens(tokens) for tokens in train_tokens.values()])
X_val = np.array([encode_tokens(tokens) for tokens in val_tokens.values()])
X_test = np.array([encode_tokens(tokens) for tokens in test_tokens.values()])

print(f"📌 Dữ liệu train shape: {X_train.shape}")
print(f"📌 Dữ liệu val shape: {X_val.shape}")
print(f"📌 Dữ liệu test shape: {X_test.shape}")


📌 Dữ liệu train shape: (225, 100, 30)
📌 Dữ liệu val shape: (57, 100, 30)
📌 Dữ liệu test shape: (342, 100, 30)


In [4]:
# 🔹 1. Mount Google Drive
drive.mount('/content/drive')

# 🔹 2. Định nghĩa đường dẫn lưu file
save_dir = "/content/drive/MyDrive/Colab Notebooks/NLP/Task DP-CNN/"
save_path = os.path.join(save_dir, "dp_cnn_embeddings.pkl")

# 🔹 3. Tạo thư mục nếu chưa tồn tại
os.makedirs(save_dir, exist_ok=True)

# 🔹 5. Lưu file vào thư mục trong Google Drive
data = {
    "X_train_embeddings": X_train,
    "X_test_embeddings": X_test,
    "X_val_embeddings": X_val
}

with open(save_path, "wb") as f:
    pickle.dump(data, f)

print(f"✅ File đã được lưu tại: {save_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ File đã được lưu tại: /content/drive/MyDrive/Colab Notebooks/NLP/Task DP-CNN/dp_cnn_embeddings.pkl
